In [2]:
import sys, os

sys.path.insert(0, "../src")
from scraper.linkedin import Head_Hunter_9000

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ChromeOptions
import html2text
from selenium.common.exceptions import NoSuchElementException
import urllib.parse
import configparser
import random
import time

import re
from data.architecture import JobBoard, Job, FreeResponseQuestion, RadioButtonsQuestion, DropDownQuestion, Option



In [3]:
hh_9000 = Head_Hunter_9000('/home/luca/Documents/Projects/Head_Hunter_9000/config.ini')
hh_9000.login()

https://www.linkedin.com/jobs/search/?f_AL=true&f_E=1%2C2%2C3&f_JT=F%2CC%2CT&f_WT=1%2C2%2C3&keywords=Data%20Analyst&location=United%20States


In [4]:
ext_job_id = hh_9000.get_job_id(hh_9000.driver.current_url)
job_info_container = hh_9000.driver.find_element(By.XPATH, "//div[@class='job-view-layout jobs-details']")
job_info = hh_9000.build_job_info(job_info_container, ext_job_id)

In [5]:
job_info

{'companyname': 'Greenline Pet',
 'location': 'Nashville, TN',
 'workplacetype': 'On-site',
 'postedtimeago': '2 days ago',
 'numapplicants': '316 applicants',
 'salary': '$70,000/yr - $80,000/yr + Bonus, Profit sharing · Full-time',
 'numemployees': '11-50 employees',
 'jobtitle': 'Data Analyst',
 'description': 'We understand that no candidate is perfectly qualified for any job. A clear\ndemonstration of accountability, a willingness to collaborate, and passion say\nmore than the words on a resume. We expect you to learn and grow in this role.\nYou are encouraged to apply even if your experience is not a “perfect” match\nfor what we’re looking for.\n\n **Position Description Summary**\n\nWe are seeking a **Data Analyst** , who will work in Greenline’s Franklin,\nTennessee office. This role will sit in the Data Operations team and report to\nthe Manager of Data and Analytics Engineering. In this position you will\nconduct analyses, report your findings to internal stakeholders, and he

In [6]:
easy_apply_button = job_info_container.find_element(By.XPATH, "//button[contains(@class, 'jobs-apply-button')]")
easy_apply_button.click()

In [ ]:
from selenium.webdriver.support.ui import Select

def get_next_btn():
    nxt_btn = hh_9000.driver.find_elements(By.XPATH, "//span[text()='Next']/ancestor::button")
    if nxt_btn:
        return nxt_btn[0]
    
    rvw_btn = hh_9000.driver.find_elements(By.XPATH, "//span[text()='Review']/ancestor::button")
    if rvw_btn:
        return rvw_btn[0]
    
    return None


def fill_out_questions(freeresponse_question_containers, dropdown_question_containers, radiobutton_question_containers):
    for fr_q in freeresponse_question_containers:
        input_tag = fr_q.find_element(By.TAG_NAME, "input")
        input_tag.clear()
        input_tag.send_keys("1")

    for dd_q in dropdown_question_containers:
        select_el = dd_q.find_element(By.TAG_NAME, "select")
        select = Select(select_el)
        select.select_by_index(1)

    for rb_q in radiobutton_question_containers:
        radio_buttons = rb_q.find_elements(By.XPATH, ".//label")
        if radio_buttons:
            radio_buttons[0].click()


fr_prompts = []
dd_prompts_and_options = []
rb_prompts_and_options = []

next_btn = True
while next_btn:
    next_btn = get_next_btn()
    question_form = hh_9000.driver.find_element(By.XPATH, "//div[@class='pb4']")

    freeresponse_question_containers = question_form.find_elements(By.XPATH, "./div[contains(@class, 'jobs-easy-apply-form-section__grouping')]//div[@data-test-single-line-text-form-component]")
    dropdown_question_containers = question_form.find_elements(By.XPATH, "./div[contains(@class, 'jobs-easy-apply-form-section__grouping')]//div[@data-test-text-entity-list-form-component]")
    radiobutton_question_containers = question_form.find_elements(By.XPATH, "./div[contains(@class, 'jobs-easy-apply-form-section__grouping')]//fieldset[@data-test-form-builder-radio-button-form-component]")

    fill_out_questions(freeresponse_question_containers, dropdown_question_containers, radiobutton_question_containers)

    fr_prompts.extend(hh_9000.scrape_freeresponse_questions(freeresponse_question_containers))
    dd_prompts_and_options.extend(hh_9000.scrape_dropdown_questions(dropdown_question_containers))
    rb_prompts_and_options.extend(hh_9000.scrape_radiobutton_questions(radiobutton_question_containers))

    next_btn.click()


In [16]:
dd_prompts_and_options

[('Email address',
  [{'text': 'Select an option', 'value': 'Select an option'},
   {'text': 'luca.guarro@gmail.com', 'value': 'luca.guarro@gmail.com'}]),
 ('Phone country code',
  [{'text': 'Select an option', 'value': 'Select an option'},
   {'text': 'United States (+1)', 'value': 'United States (+1)'},
   {'text': 'Afghanistan (+93)', 'value': 'Afghanistan (+93)'},
   {'text': 'Aland Islands (+358)', 'value': 'Aland Islands (+358)'},
   {'text': 'Albania (+355)', 'value': 'Albania (+355)'},
   {'text': 'Algeria (+213)', 'value': 'Algeria (+213)'},
   {'text': 'American Samoa (+1)', 'value': 'American Samoa (+1)'},
   {'text': 'Andorra (+376)', 'value': 'Andorra (+376)'},
   {'text': 'Angola (+244)', 'value': 'Angola (+244)'},
   {'text': 'Anguilla (+1)', 'value': 'Anguilla (+1)'},
   {'text': 'Antarctica (+0)', 'value': 'Antarctica (+0)'},
   {'text': 'Antigua and Barbuda (+1)', 'value': 'Antigua and Barbuda (+1)'},
   {'text': 'Argentina (+54)', 'value': 'Argentina (+54)'},
   {'te

In [15]:
rb_prompts_and_options

[("Have you completed the following level of education: Bachelor's Degree?",
  [{'text': 'Yes', 'value': 'Yes'}, {'text': 'No', 'value': 'No'}]),
 ("Have you completed the following level of education: Bachelor's Degree?",
  [{'text': 'Yes', 'value': 'Yes'}, {'text': 'No', 'value': 'No'}]),
 ('Will you now, or in the future, require sponsorship for employment visa status (e.g. H-1B visa status)?',
  [{'text': 'Yes', 'value': 'Yes'}, {'text': 'No', 'value': 'No'}])]

In [31]:
for rb_q_c in radiobutton_question_containers:
    question_prompt = rb_q_c.find_element(By.XPATH, "//span[@data-test-form-builder-radio-button-form-component__title]/span[@aria-hidden='true']").text
    print(question_prompt)

    input_containers = rb_q_c.find_elements(By.TAG_NAME, "div")

    # List to store the dictionaries
    option_list = []

    for input_container in input_containers:
        input = input_container.find_element(By.TAG_NAME, "input")
        value = input.get_attribute('value')

        inner_text = input_container.find_element(By.TAG_NAME, "label").text

        option_dict = {
            "text": inner_text,
            "value": value
        }
        option_list.append(option_dict)

Are you legally authorized to work in the United States?


In [32]:
option_list

[{'text': 'Yes', 'value': 'penis'}, {'text': 'No', 'value': 'No'}]

In [26]:
for dd_q_c in dropdown_question_containers:
    question_prompt = dd_q_c.find_element(By.XPATH, "./label/span[@aria-hidden='true']").text

    select = dd_q_c.find_element(By.TAG_NAME, "select")
    options = select.find_elements(By.TAG_NAME, "option")

    # List to store the dictionaries
    option_list = []

    # Loop through the "option" elements and extract inner text and value
    for option in options:
        inner_text = option.text
        value = option.get_attribute("value")

        # Create a dictionary for each "option" element and add it to the list
        option_dict = {
            "text": inner_text,
            "value": value
        }
        option_list.append(option_dict)

    # Print the list of dictionaries
    print(question_prompt)
    print(option_list)

This is an in-person role in Horizon's Charlotte, NC office. If you don't currently live in the Charlotte area, are you open to relocating?
[{'text': 'Select an option', 'value': 'Select an option'}, {'text': 'Yes', 'value': 'Yes'}, {'text': 'No', 'value': 'No'}]


In [27]:
next_btn = hh_9000.driver.find_element(By.XPATH, "//span[text()='Next']/ancestor::button")
next_btn.click()

In [8]:
job_info_container = hh_9000.driver.find_element(By.XPATH, "//div[@class='job-view-layout jobs-details']")

In [9]:
job_short = job_info_container.find_element(By.XPATH, ".//div[@class='jobs-unified-top-card__content--two-pane']")
company_location_info = job_short.find_element(By.XPATH, "./div[@class='jobs-unified-top-card__primary-description']")

In [10]:
company_location_info.text

'Heartland America · Chaska, MN (Hybrid)  1 day ago  · 378 applicants'

In [8]:
job_info = {}
hh_9000.add_info_if_exists(job_info, 'salary', job_short, "(.//li[@class='jobs-unified-top-card__job-insight'])[1]")
hh_9000.add_info_if_exists(job_info, 'numemployees', job_short, "(.//li[@class='jobs-unified-top-card__job-insight'])[2]")
hh_9000.add_info_if_exists(job_info, 'location', company_location_info, "./div")
hh_9000.add_info_if_exists(job_info, 'workplacetype', company_location_info, "./div")

In [9]:
job_info

{'salary': 'Full-time',
 'numemployees': '1-10 employees',
 'location': 'Dirac, Inc. · New York City Metropolitan Area (Hybrid)  3 weeks ago  · 280 applicants',
 'workplacetype': 'Dirac, Inc. · New York City Metropolitan Area (Hybrid)  3 weeks ago  · 280 applicants'}

In [14]:
def parse_job_string(job_string):
    pattern = r'^(.*?) · (.*?) \((.*?)\) +(\d+ (day|week|month)s? ago) +· +(\d+(?:,\d+)? applicants)$'
    match = re.match(pattern, job_string)

    if match:
        company_name = match.group(1)
        location = match.group(2)
        workplace_type = match.group(3)
        posted_time_ago = match.group(4)
        num_applicants = match.group(6).replace(',', '')

        job_attributes = {
            "company_name": company_name,
            "location": location,
            "workplace_type": workplace_type,
            "posted_time_ago": posted_time_ago,
            "num_applicants": num_applicants
        }
        return job_attributes
    else:
        return None
    
job_attr_from_string = parse_job_string("Heartland America · Chaska, MN (Hybrid)  1 day ago  · 378 applicants")

In [15]:
job_attr_from_string

{'company_name': 'Heartland America',
 'location': 'Chaska, MN',
 'workplace_type': 'Hybrid',
 'posted_time_ago': '1 day ago',
 'num_applicants': '378 applicants'}

In [20]:
job_dict = {}
job_dict.update(job_attr_from_string)

In [21]:
job_dict

{'company_name': 'Dirac, Inc.',
 'location': 'New York City Metropolitan Area',
 'workplace_type': 'Hybrid',
 'posted_time_ago': '3 weeks',
 'num_applicants': '280 applicants'}

In [4]:
time.sleep(random.uniform(1,2))
hh_9000.scroll_through_sidebar()
job_listings = hh_9000.driver.find_elements_by_xpath("//div[contains(@class, 'job-card-container') and contains(@class, 'job-card-list')]")

In [14]:
from selenium.common.exceptions import NoSuchElementException

In [17]:
def loop_through_questions(hh_9000, job_id):
    job_info_container = hh_9000.driver.find_element_by_xpath("//div[@class='job-view-layout jobs-details']")
    job_short = job_info_container.find_element_by_xpath(".//div[@class='jobs-unified-top-card__content--two-pane']")
    job_short.find_element_by_xpath(".//button[contains(@class, 'jobs-apply-button')]").click() # click easy apply
    # Loop through questions  //button[@data-easy-apply-next-button]
    more_questions = True
    while(more_questions):
        dd_and_fr_question_forms = hh_9000.driver.find_elements_by_xpath("//div[@class='jobs-easy-apply-content']//div[@class='jobs-easy-apply-form-section__grouping']/div[contains(@class, 'jobs-easy-apply-form-element')]")
        rb_question_forms = hh_9000.driver.find_elements_by_xpath("//div[@class='jobs-easy-apply-content']//div[@class='jobs-easy-apply-form-section__grouping']/fieldset")

        for q_form in dd_and_fr_question_forms: # dropdown: fb-dropdown    ---   free-response: fb-single-line-text
            question_text = q_form.find_element_by_xpath("./label/span[not(@class='visually-hidden')]").text
            question_type = q_form.find_element_by_xpath("./div").get_attribute("class")

            # //div[@class='jobs-easy-apply-content']//div[@class='jobs-easy-apply-form-section__grouping']/div[contains(@class, 'jobs-easy-apply-form-element')]/div/select/option
            if question_type == "fb-dropdown":
                
                options = q_form.find_elements_by_xpath("./div/select/option")
                option_vals = [option.get_attribute("value") for option in options]

                option_sets = hh_9000.litehelper.get_from_table("Questions q INNER JOIN DropDownQuestions d on q.QuestionID = d.DropDownQuestionID", ['OptionSetID'], {"Question": question_text})
                if option_sets: # question has already been added
                    for option_set in option_sets:
                        db_options = hh_9000.litehelper.get_from_table("Options", ['OptionText', 'Value'], {"OptionSetID": option_set})
                        db_option_texts = set()
                        if all([option_text in db_option_texts for option_text in option_texts]) and all([option_val in db_option_vals for option_val in option_vals]):
                            q_id = hh_9000.litehelper.get_from_table("DropDownQuestions", ['DropDownQuestionID'], {"OptionSet": option_set})
                            hh_9000.litehelper.insert_job_question(**{'JobID': job_id, 'QuestionID': q_id})
                            break
                    
                        
                    

                # if question is in db
                    # store all option_sets in a list of sets
                    # then for each set check if all options in the UI are in that set
                    # if ALL options are in that set, then this is the correct set and we just need to store reference to the OptionSetID
                    # else we need to add the same question to the db - this time with the new option set
                # else we need to add it to the db along with all the options
                
                

            print(question_text, question_type)

        for q_form in rb_question_forms:
            question_text = q_form.find_element_by_xpath("./legend/span[not(@class='visually-hidden')]").text
            options = q_form.find_element_by_xpath("./div[@class='fb-radio-buttons']")
            print(question_text)

        next_q_button = find_element_by_xpath_safe(hh_9000.driver, "//button[@data-easy-apply-next-button]")
        if next_q_button:
            next_q_button.click()
        else:
            more_questions = False

def find_element_by_xpath_safe(element, xpath_from_element):
    try:
        return element.find_element_by_xpath(xpath_from_element)
    except NoSuchElementException:
        return None

In [18]:
for i in range(len(job_listings)):
    just_added = False
    time.sleep(random.uniform(1,2))
    link = job_listings[i].find_element_by_xpath(".//a[contains(@class, 'job-card-container__link') and contains(@class, 'job-card-list__title')]")
    link.click()
    ext_job_id = job_listings[i].get_attribute("data-job-id")
    if not hh_9000.litehelper.is_in_table('Jobs', **{'JobBoardID':'linkedin', 'ExtJobID': ext_job_id}):
        job_info = hh_9000.build_job_info(ext_job_id)
        hh_9000.litehelper.insert_job(**job_info)
        just_added = True

    if just_added or hh_9000.litehelper.is_in_table('Jobs', **{'JobBoardID':'linkedin', 'ExtJobID': ext_job_id, 'AppSubmitted': 0}):
        loop_through_questions(hh_9000)
        break

            # If all Questions for Job already have Answers

            # Else if there are not yet any Questions

Email address fb-dropdown
Phone country code fb-dropdown
Mobile phone number fb-single-line-text
How many years of work experience do you have using Tableau? fb-single-line-text
How many years of work experience do you have using Python (Programming Language)? fb-single-line-text
Have you completed the following level of education: Bachelor's Degree?
Are you legally authorized to work in the United States?


### Old

In [5]:
job_listings = hh_9000.driver.find_elements_by_xpath("//div[contains(@class, 'job-card-container') and contains(@class, 'job-card-list')]")

In [8]:
job_info_container = hh_9000.driver.find_element_by_xpath("//div[@class='job-view-layout jobs-details']")
job_short = job_info_container.find_element_by_xpath(".//div[@class='jobs-unified-top-card__content--two-pane']")
easy_apply = job_short.find_element_by_xpath(".//button[contains(@class, 'jobs-apply-button')]").click()

In [25]:
dd_and_fr_question_forms = hh_9000.driver.find_elements_by_xpath("//div[@class='jobs-easy-apply-content']//div[@class='jobs-easy-apply-form-section__grouping']/div[contains(@class, 'jobs-easy-apply-form-element')]")
rb_question_forms = hh_9000.driver.find_elements_by_xpath("//div[@class='jobs-easy-apply-content']//div[@class='jobs-easy-apply-form-section__grouping']/fieldset")

In [26]:
for q_form in dd_and_fr_question_forms:
    question_text = q_form.find_element_by_xpath("./label/span[not(@class='visually-hidden')]").text
    question_type = q_form.find_element_by_xpath("./div").get_attribute("class")
    print(question_text, question_type)

In [27]:
for q_form in rb_question_forms:
    question_text = q_form.find_element_by_xpath("./legend/span[not(@class='visually-hidden')]")
    options = q_form.find_element_by_xpath("./div[@class='fb-radio-buttons']")
    print(question_text)

Are you comfortable commuting to this job's location?
